This notebook will develope tools to calculate the pressure from the hydrostatic relation. 

In [1]:
import netCDF4 as nc
import numpy as np
import matplotlib.pyplot as plt

from salishsea_tools import viz_tools, psu_tools, grid_tools

import gsw

%matplotlib inline

Plan: 
* Use hydrostatic approximation to calculate pressure:
$ \frac{\partial p}{\partial z} = -\rho g $ 
with $p(z=0) = 0$
* For now, calculate density with the function Susan gave me. How should this change with TEOS-10?
* To enforce $p(z=0)=0$, I think it is best to interpolate rho onto w grid and then integrate for pressure with upper BC and then interpolate back to t grid 

# Loading

In [2]:
mesh_mask = nc.Dataset('/data/nsoontie/MEOPAR/NEMO-forcing/grid/mesh_mask_SalishSea2.nc')
grid = nc.Dataset('/data/nsoontie/MEOPAR/NEMO-forcing/grid/bathy_meter_SalishSea2.nc')

bathy = grid.variables['Bathymetry'][:]
# Sample file for testing calculations
fT = nc.Dataset('/results/SalishSea/nowcast/01apr15/SalishSea_1h_20150401_20150401_grid_T.nc')

In [3]:
tmask = mesh_mask.variables['tmask'][:]
e3t_0 = mesh_mask.variables['e3t'][:]
gdept_0 = mesh_mask.variables['gdept'][:]
gdepw_0 = mesh_mask.variables['gdepw'][:]

In [4]:
ssh = fT.variables['sossheig'][:]
sal = fT.variables['vosaline'][:]
temp = fT.variables['votemper'][:]
rho = psu_tools.calculate_density(temp, sal)

In [6]:
grids = grid_tools.calculate_vertical_grids(e3t_0[0,...], tmask[0,...], ssh)

/data/nsoontie/MEOPAR/tools/SalishSeaTools/salishsea_tools/grid_tools.py:48: RuntimeWarning: invalid value encountered in true_divide
  mu = sum_matrix/vn


In [7]:
def calculate_pressure(rho, e3t_t, gdept_t, tmask):
    """Integrate density to calulate pressure.
    Use time-dependent scale factors
    
    returns pressure in Pascals"""
    g = 9.81 # is there a constant module that I an grab this from?
    
    # integrate density to get pressure
    p = np.cumsum(g*rho*e3t_t*tmask, axis=1)
    p = p - np.expand_dims(g*rho[:,0,...]*gdept_t[:,0,...], axis=1) # Attempt to approximate p(z=0)=0.
    
    return p
    

In [8]:
t=2
p_t = calculate_pressure(rho[0:t], grids['e3t_t'][0:t], grids['gdept_t'][0:t], tmask)

In [9]:
p_t[0,:,400,300]

array([   5081.97537416,   15264.71065816,   25504.05833731,
         35771.60396324,   46046.96165661,   56320.31685063,
         66586.84866391,   76844.68407262,   87094.80132038,
         97340.56650632,  107589.17846981,  117855.13098334,
        128167.18247194,  138581.89624688,  149209.30717496,
        160262.41149617,  172151.62408805,  185664.02926869,
        202286.49356069,  224749.78986749,  232328.09970387,
        232328.09970387,  232328.09970387,  232328.09970387,
        232328.09970387,  232328.09970387,  232328.09970387,
        232328.09970387,  232328.09970387,  232328.09970387,
        232328.09970387,  232328.09970387,  232328.09970387,
        232328.09970387,  232328.09970387,  232328.09970387,
        232328.09970387,  232328.09970387,  232328.09970387,
        232328.09970387])

In [10]:
lat = fT.variables['nav_lon'][:]
p_gsw = gsw.p_from_z(-grids['gdept_t'][0:t], lat)
p_gsw[0,:,400,300]

array([   0.52108837,    1.56256533,    2.60264199,    3.6413239 ,
          4.67862189,    5.71455702,    6.74917031,    7.78254152,
          8.81482584,    9.84632512,   10.87762643,   11.9098714 ,
         12.94527957,   13.98816428,   15.04690083,   16.13772511,
         17.29201181,   18.57001916,   20.08611348,   22.05246076,
         23.56380259,   24.3259087 ,   28.73143521,   38.72458044,
         52.8235531 ,   71.0291951 ,   92.62161427,  116.58029047,
        142.00115683,  168.25650037,  194.96672563,  221.91924366,
        248.99982402,  276.14841366,  303.33379866,  330.53984988,
        357.75825781,  384.98474782,  412.2171223 ,  439.45425157])

In [11]:
diff = (p_t*10**(-4) - p_gsw)*tmask

In [12]:
diff[0,:,400,300]

array([-0.01289083, -0.03609426, -0.05223616, -0.06416351, -0.07392572,
       -0.08252534, -0.09048544, -0.09807311, -0.10534571, -0.11226847,
       -0.11870859, -0.1243583 , -0.12856132, -0.12997466, -0.12597011,
       -0.11148396, -0.07684941, -0.00361623,  0.14253588,  0.42251823,
       -0.33099262, -0.        , -0.        , -0.        , -0.        ,
       -0.        , -0.        , -0.        , -0.        , -0.        ,
       -0.        , -0.        , -0.        , -0.        , -0.        ,
       -0.        , -0.        , -0.        , -0.        , -0.        ])